# [LVV-T2214] _ Plots

This notebook will analyze the data collected when running LVV-T2214 test case during L3 integration tests.  

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

Requirements:

    You should have run this test case and record the time stamps on each step.

Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.

Make sure you run this notebook on TTS before running at the summit.

Please, see the README file for the requirements to run this notebook.

In [ ]:
test_case = "LVV-T2214"
test_exec = "LVV-D28Jul"

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from astropy.time import Time,TimeDelta
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.sitcom import vandv

In [ ]:
client = vandv.efd.create_efd_client()

exec_info = vandv.ExecutionInfo()
print(exec_info)

---- 
## Define time windows for the test execution 
    # Edit with date times. 
    Gathering data - without Aberrations - Start time: 2022-06-21T00:36:46.077
    Gathering data - without Aberrations - End time: 2022-06-21T00:41:25.742

    Gathering data - with Aberrations (reset every time) - Start time: 2022-06-21T01:51:30.700
    Gathering data - with Aberrations (reset every time) - End time: 2022-06-21T01:56:32.971

    Gathering data - with Aberrations Accumulated - Start time: 2022-06-21T01:57:36.178
    Gathering data - with Aberrations Accumulated - End time: 2022-06-21T02:02:26.898

In [ ]:
# Without aberrations
t_start = "2022-06-21T00:36:46.077"
t_start = Time(t_start, format="isot", scale="utc")

t_end = "2022-06-21T00:41:25.742"
t_end = Time(t_end, format="isot", scale="utc")

Run the cell below that correspond to the test you want to analyze in this notebook (Declare only one of them) 
- With aberrations (Reset every time)
- With aberrations (Accumulated)

In [ ]:
# With aberrations (Reset every time)
test_type = "With aberrations (Reset every time)"
t_start_aberr = "2022-06-21T01:51:30.700"
t_start_aberr = Time(t_start_aberr, format="isot", scale="utc")

t_end_aberr = "2022-06-21T01:56:32.971"
t_end_aberr = Time(t_end_aberr, format="isot", scale="utc")

reset = 2

In [ ]:
# With aberrations (Accumulated) 
test_type = "With aberrations (Accumulated)"
t_start_aberr_accum = "2022-06-21T01:57:36.178"
t_start_aberr = Time(t_start_aberr_accum, format="isot", scale="utc")

t_end_aberr_accum = "2022-07-28T18:54:18.825"
t_end_aberr = Time(t_end_aberr_accum, format="isot", scale="utc")

reset = 1

---
## Helper Functions

In [ ]:
async def get_data_from_efd(start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """
    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start, 
        end=end,
    )
    
    _df_m1m3_z28 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="zForce28",
        start=start, 
        end=end,
    )
        
    # M2 B1?
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
            "lutGravity0",
            "measured0"
        ],
        start=start, 
        end=end,
    )
    
    # CamHex Z position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position2"
        ],
        index=1,
        start=start, 
        end=end,
    )
    
    # M2Hex Z position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position2"
        ],
        index=2,
        start=start, 
        end=end,
    )

    # Rename columns
    _df_mount_el.rename(columns={"actualPosition": "mount_el"}, inplace=True)
    _df_m1m3_z28.rename(columns={"zForce28": "m1m3_z28"}, inplace=True)
    _df_m2.rename(columns={"applied0": "m2b1_applied", "lutGravity0": "m2b1_gravLut", "measured0": "m2b1_measured"}, inplace=True)
    _df_camhex.rename(columns={"position2": "camhex_z"}, inplace=True)
    _df_m2hex.rename(columns={"position2": "m2hex_z"}, inplace=True)

    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3_z28, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
         
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

In [ ]:
async def get_track_info(start_time, end_time):
    
    """
    Retrieves tracking information for each slew returning StartTracking, StopTracking and moveToTarget times.
    
    Parameters
    ----------
    start_time : str or datetime
        Start of the time-window.
    end_time : str or datetime 
        End of the time-window.
    """
    
    moveToTarget_times = []
    
    startTracking_times = await client.select_time_series(
        'lsst.sal.MTMount.command_startTracking', 
        ['*'], 
        start_time, 
        end_time
    )

    stopTracking_times = await client.select_time_series(
        'lsst.sal.MTMount.command_stopTracking', 
        ['*'], 
        start_time, 
        end_time
    )

    _move = await client.select_time_series(
        'lsst.sal.MTMount.command_moveToTarget', 
        ['*'], 
        start_time, 
        end_time
    )  

    for track in tracks:
        # Find the time of the first moveToTarget command for each of the slews
        if track ==0:
            move_time = _move[(_move.index <= startTracking_times.index[track])].index[0]
        else: 
            move_time = _move[(_move.index >= stopTracking_times.index[track-1]) 
                                        & (_move.index <= startTracking_times.index[track])].index[0]
        
        moveToTarget_times.append(move_time)
        
    return startTracking_times, stopTracking_times, moveToTarget_times

In [ ]:
async def build_m1m3_snapshot_series(start, end):
    fel = await client.select_time_series(
        "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
        "*", 
        start,
        end)

    fba = await client.select_time_series(
        "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
        "*",
        start,
        end)

    fst = await client.select_time_series(
        "lsst.sal.MTM1M3.logevent_appliedStaticForces",
        "*",
        start,
        end)

    fao = await client.select_time_series(
        "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
        "*", 
        start,
        end)
    
    fad = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData", 
        "*", 
        start,
        end)
    
    el = await client.select_time_series(
        "lsst.sal.MTMount.elevation",
        "*", 
        start,
        end)

    labels = [
        "appliedElevationForces", 
        "appliedBalanceForces", 
        "appliedStaticForces",
        "appliedActiveOpticForces",
        "ForceActuatorData"
    ]
    
    return fel,fba,fst,fao,fad,el,labels

In [ ]:
async def build_m2_snapshot_series(start, end):

    axf = await client.select_time_series(
        "lsst.sal.MTM2.axialForce", 
        "*",
        start,
        end)

    taf = await client.select_time_series(
        "lsst.sal.MTM2.tangentForce", 
        "*",
        start,
        end)

    cof = await client.select_time_series(
        "lsst.sal.MTM2.command_applyForces",
        "*", 
        start,
        end)
    
    el = await client.select_time_series(
        "lsst.sal.MTMount.elevation",
        "*", 
        start,
        end)
    
    labels = cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
    ]
    
    
    return axf, taf, cof, el, labels 

In [ ]:
async def get_hex_positions(start,end, ind): # From the XML:
    #   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
    #   Linear positions are in microns, angular positions are in degrees.
    pos = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        "*",
        start,
        end,
        index=ind
    )

    # Unravel in x/y/z/u/v/w
    for i, col in enumerate("xyzuvw"):
        pos[col] = pos[f"position{i}"]


    # Triggered at the end of a slew
    cpos = await client.select_time_series(
        "lsst.sal.MTHexapod.logevent_compensatedPosition",
        "*",
        start,
        end,
        index=ind
    )

    # Triggered only after move/offset. Should not see much. 
    upos = await client.select_time_series(
        "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
        "*",
        start,
        end,
        index=ind
    ) 
    
    el = await client.select_time_series(
        "lsst.sal.MTMount.elevation",
        "*", 
        start,
        end)
    
    return pos, cpos, upos, el

----
## Define tracks and get information for each track (MoveToTarget, StartTracking, StopTracking) and the times at which MTAOS corrections were issued. 

In [ ]:
tracks = [0,1,2,3]
aberrations = ['Focus', 'Astigmatism', 'Coma', 'Trefoil']

The cell below gets the required information for each track from the mount, such as when it started tracking, stopped tracking and when was it commanded to slew (first moveToTarget) 

In [ ]:
startTracking_without, stopTracking_without, move_without = await get_track_info(t_start, t_end)
startTracking_withaberr, stopTracking_withaberr, move_withaberr = await get_track_info(t_start_aberr, t_end_aberr)

Querying EFD times at which MTAOS corrections were issued in the test where aberrations were commanded. 

In [ ]:
corrections_issued = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m1m3Correction', 
    ["zForces28"],       
    t_start_aberr, 
    t_end_aberr
)

---
## Plot optics vs time for each test. 

Query telemetry

In [ ]:
df_without = await get_data_from_efd(t_start, t_end)

df_aberr = await get_data_from_efd(t_start_aberr, t_end_aberr)


In [ ]:
# Without aberrations
fig, axs = plt.subplots(figsize=(20, 12.5), nrows=5, sharex=True)

axs[0].plot(df_without["mount_el"].dropna(), "C6o-", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_without["m1m3_z28"].dropna(), "C7*-", label="M1M3 zForce28")
axs[1].set_ylabel("M1M3 z28 \n Force [N]")
 
axs[2].plot(df_without["m2b1_measured"].dropna(), "C8v-", label="M2 B1 Measured")
axs[2].set_ylabel("M2 B1 Measured \n Force [N]")

axs[3].plot(df_without["camhex_z"].dropna(), "C4x-", label="CamHex Z")
axs[3].set_ylabel("Cam Hexapod \n Z Position [um]")

axs[4].plot(df_without["m2hex_z"].dropna(), "C5+-", label="M2Hex Z")
axs[4].set_ylabel("M2 Hexapod \n Z Position [um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()
    for k in np.arange(len(startTracking_without)):
        l_start = ax.axvline(x=startTracking_without.index[k],c="green",linewidth=2,zorder=0, clip_on=False, ls = "-.")
        l_stop = ax.axvline(x=stopTracking_without.index[k],c="red",linewidth=2, zorder=0,clip_on=False, ls = "-.")
        l_move = ax.axvline(x=(move_without[k]), c="blue", ls = ":")
        
fig.legend([l_start,l_stop,l_move],['StartTracking','StopTracking','MoveToTarget'], 
           ncol=3, loc='upper right', bbox_to_anchor=(1.0, 0.99))

fig.suptitle(f"{test_exec} - M1M3/M2/Hexs/Elevation vs Time - No AOS aberrations added")
fig.tight_layout(h_pad=0.5)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_m1m3_m2_hexs_el_vs_time_without.png")
plt.show()

In [ ]:
# With aberration

fig, axs = plt.subplots(figsize=(20, 12.5), nrows=5, sharex=True)

axs[0].plot(df_aberr["mount_el"].dropna(), "C6o-", label="Mount Elevation Aberr")
axs[0].set_ylabel("Mount El\n[deg]")

axs[0].text(startTracking_withaberr.index[0]+timedelta(seconds=15), 83, 'Focus')
axs[0].text(startTracking_withaberr.index[1]+timedelta(seconds=10), 83, 'Astigmatism')
axs[0].text(startTracking_withaberr.index[2]+timedelta(seconds=10), 83, 'Y-Coma')
axs[0].text(startTracking_withaberr.index[3]+timedelta(seconds=12), 83, 'Trefoil')

axs[1].plot(df_aberr["m1m3_z28"].dropna(), "C7*-", label="M1M3 zForce28 Aberr")
axs[1].set_ylabel("M1M3 z28 \n Force [N]")

axs[2].plot(df_aberr["m2b1_measured"].dropna(), "C8v-", label="M2 B1 Measured Aberr")
axs[2].set_ylabel("M2 B1 Measured \n Force [N]")

axs[3].plot(df_aberr["camhex_z"].dropna(), "C4x-", label="CamHex Z Aberr")
axs[3].set_ylabel("Cam Hexapod \n Z Position [um]")

axs[4].plot(df_aberr["m2hex_z"].dropna(), "C5+-", label="M2Hex Z Aberr")
axs[4].set_ylabel("M2 Hexapod \n Z Position [um]")


for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()
    for track in np.arange(len(startTracking_without)):
        l_start = ax.axvline(x=startTracking_withaberr.index[track],c="green", ls = "-.")
        l_stop = ax.axvline(x=stopTracking_withaberr.index[track],c="red",ls = "-.")
        l_move = ax.axvline(x=(move_withaberr[track]), c="blue", ls = ":")
        l_aos = ax.axvline(x=corrections_issued.index[track*reset],c="cyan",ls = "--")
        
fig.legend([l_start,l_stop,l_move,l_aos],['StartTracking','StopTracking','MoveToTarget','ApplyCorrections'], ncol=4, loc='upper right', bbox_to_anchor=(1.0, 0.99))


fig.suptitle(f"{test_exec} - M1M3/M2/Hexs/Elevation with AOS added aberrations")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

---
## Compare against the corrections sent by MTAOS. 

check the force and hexapod position differences between steps 10 and 12.

Compare against the corrections sent by MTAOS. They should match.

Get the forces and hexapod positions approx. 1 second before moving to target for each track with 
and without aberrations. 

---
### M1 forces vs MTAOS corrections

Get M1M3 corrections

In [ ]:
m1m3_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m1m3Correction', 
    [f"zForces{i}" for i in range(156)],       
    t_start_aberr, 
    t_end_aberr
)
m1m3_correction.columns = m1m3_correction.columns.str.replace('zForces', 'zForce')

Get data 


In [ ]:
fel_aberr, fba_aberr, fst_aberr, fao_aberr, fad_aberr, el_aberr, labels_aberr = await build_m1m3_snapshot_series(t_start_aberr, t_end_aberr)
fel, fba, fst, fao, fad, el, labels = await build_m1m3_snapshot_series(t_start, t_end)


M1M3 Timeline

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew Without Correction - Snapshot Forces", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 3),
)

fig.suptitle("M1M3 Timeline - Without Aberrations")
_ = vandv.m1m3.timeline_zforces(axs["A"], fad, "zForce", elevation=el)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew With Correction - Snapshot Forces", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 3),
)

fig.suptitle("M1M3 Timeline - With Aberrations")
_ = vandv.m1m3.timeline_zforces(axs["A"], fad_aberr, "zForce", elevation=el_aberr)

M1M3 Forces without and with Aberrations vs MTAOS corrections

In [ ]:
for track,aberr in zip(tracks,aberrations):

    fig, axs = plt.subplot_mosaic(
    mosaic="ABCD\nEFGH\nIJKL",
    num="Slew Without Correction - Snapshot zForces Overview", 
    constrained_layout=False,
    dpi=120,
    figsize=(15, 10)
    )

    fig.suptitle(f"{test_exec} M1M3 Forces without and with 1um {aberr} aberration. \n Force difference vs MTAOS corrections")

    #With aberration

    series_aberr = [df_aberr[df_aberr.index < move_withaberr[track]].iloc[-10] for df_aberr in [fel_aberr, fba_aberr, fst_aberr, fao_aberr]]

    for ax, s, label in zip("EFGH", series_aberr, labels_aberr):
        _ = vandv.m1m3.snapshot_zforces_overview(axs[ax], s, title=f'with added 1 um of {aberr}')

    # Without 

    series = [df[df.index < move_without[track]].iloc[-10] for df in [fel, fba, fst, fao]]

    for ax, s, label in zip("ABCD", series, labels):
        _ = vandv.m1m3.snapshot_zforces_overview(axs[ax], s, title=f'{label} \n with no aberration')
        
    # Plot the m1m3 correction values 
    
    _ = vandv.m1m3.snapshot_zforces_overview(axs['I'], m1m3_correction.iloc[track*reset], title = 'MTAOS commanded \n corrections')
    
    # Plot the difference in Actuator Data between the tracks with aberration and without
    
    forces_aberr = fad_aberr[fad_aberr.index < move_withaberr[track]].iloc[-10]  
    forces_without = fad[fad.index < move_without[track]].iloc[-10]
    
    cols = [c for c in forces_without.index if 'zForce' in c]
    
    difference_actuators = forces_aberr[cols] - forces_without[cols]

            
    _ = vandv.m1m3.snapshot_zforces_overview(axs['J'], difference_actuators, title = 'Actual Force Actuator Difference \n (w - wo) Aberration')

    # Difference between commanded correction and the actuator difference. 
    
    
    diff = difference_actuators[cols] - m1m3_correction.iloc[track*reset][cols]

    _ = vandv.m1m3.snapshot_zforces_overview(axs['K'], diff, title = '(Actual - Commanded) ')
    
    
    diff_perc = (difference_actuators[cols] - m1m3_correction.iloc[track*reset][cols])/m1m3_correction.iloc[track*reset][cols]

    _ = vandv.m1m3.snapshot_zforces_overview(axs['L'], diff_perc, title = '(Actual - Commanded)/Commanded \n [%]')
            
    plt.show()

We compare numerically actuators 129 and 101.

---
### M2 forces vs MTAOS corrections

Get corrections

In [ ]:
m2_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2Correction', 
    [f"zForces{i}" for i in range(72)], 
    t_start_aberr, 
    t_end_aberr
)
# Change column names to match that of commanded forces, aka axial. 
m2_correction.columns = m2_correction.columns.str.replace("zForces", "measured")

Get M2 force data

In [ ]:
axf, taf, cof, el, labels = await build_m2_snapshot_series(t_start, t_end)
axf_aberr, taf_aberr, cof_aberr, el_aberr, labels_aberr = await build_m2_snapshot_series(t_start_aberr, t_end_aberr)

M2 Timelines

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD\nE\nF\nG",
    num="M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 12),
)

fig.suptitle("M2 Timeline  - Without Aberrations")
_ = vandv.m2.timeline_axial_forces(axs["A"], axf, elevation=el)
_ = vandv.m2.timeline_axial_forces(axs["B"], axf, elevation=el, column="applied")
_ = vandv.m2.timeline_axial_forces(axs["C"], axf, elevation=el, column="lutGravity")

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf, elevation=el, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["E"], axf, elevation=el, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["F"], axf, elevation=el, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["G"], axf, elevation=el, act="B24", cols=cols)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD\nE\nF\nG",
    num="M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 12),
)

fig.suptitle("M2 Timeline  - Per force - Without Aberrations")
_ = vandv.m2.timeline_axial_forces(axs["A"], axf_aberr, elevation=el_aberr)
_ = vandv.m2.timeline_axial_forces(axs["B"], axf_aberr, elevation=el_aberr, column="applied")
_ = vandv.m2.timeline_axial_forces(axs["C"], axf_aberr, elevation=el_aberr, column="lutGravity")

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf_aberr, elevation=el_aberr, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["E"], axf_aberr, elevation=el_aberr, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["F"], axf_aberr, elevation=el_aberr, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["G"], axf_aberr, elevation=el_aberr, act="B24", cols=cols)

M2 Forces without and with Aberrations vs MTAOS corrections

In [ ]:
for track,aberr in zip(tracks,aberrations):

    fig, axs = plt.subplot_mosaic(
    mosaic="ABC\nDEF\nGHI",
    num="Slew Without Correction - M2 All Actuators", 
    constrained_layout=False,
    dpi=120,
    figsize=(12, 8),
    )
    fig.suptitle(f"{test_exec} M2 Forces without and with 1um {aberr} aberration. Force difference vs MTAOS corrections")

    #With aberration

    axf_aberr_track = axf_aberr[axf_aberr.index < move_withaberr[track]].iloc[0]
    _ = vandv.m2.snapshot_zforces_overview(axs["D"], axf_aberr_track, ms=150, fs=6, title = f"With 1 um {aberr}")
    _ = vandv.m2.snapshot_zforces_overview(axs["E"], axf_aberr_track, prefix="lutGravity", ms=150, fs=6,title = f"With 1 um {aberr}")
    _ = vandv.m2.snapshot_zforces_overview(axs["F"], axf_aberr_track, prefix="applied", ms=150, fs=6,title = f"With 1 um {aberr}")
    

    # Without 

    axf_track = axf[axf.index < move_without[track]].iloc[0]

    _ = vandv.m2.snapshot_zforces_overview(axs["A"], axf_track, ms=150, fs=6, title = "Measured axial Forces \n without Aberration")
    _ = vandv.m2.snapshot_zforces_overview(axs["B"], axf_track, prefix="lutGravity", ms=150, fs=6, title ="LutGravity Axial Forces \n without Aberration")
    _ = vandv.m2.snapshot_zforces_overview(axs["C"], axf_track, prefix="applied", ms=150, fs=6, title = "Applied Axial Forces \n without Aberration")
    
    
    # MTAOS corrections
    _ = vandv.m2.snapshot_zforces_overview(axs["G"], m2_correction.iloc[track*reset], ms=150, fs=6, title = "MTAOS corrections")
    
    # Measured Force
    measured = [c for c in axf_track.index if 'measured' in c]
    force_diff = axf_aberr_track[measured] - axf_track[measured]
    
    _ = vandv.m2.snapshot_zforces_overview(axs["H"], force_diff, ms=150, fs=6, title = "Actual Force Actuator Difference \n (w - wo) Aberration")

    # Actual Force Difference - Commanded 
    
    diff = force_diff - m2_correction.iloc[track*reset] 
    _ = vandv.m2.snapshot_zforces_overview(axs["I"], force_diff, ms=150, fs=6, title = "Actual - commanded")


    plt.show()


In [ ]:
    axf_aberr_track = axf_aberr[axf_aberr.index < move_withaberr[track]]


---
### CamHex vs MTAOS corrections

Get position data

In [ ]:
pos, cpos, upos, el = await get_hex_positions(t_start, t_end, ind=1)
pos_aberr, cpos_aberr, upos_aberr, el_aberr = await get_hex_positions(t_start_aberr, t_end_aberr, ind=1)

In [ ]:
Get the MTAOS corrections 


In [ ]:
cam_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "u", "v" , "w"], 
    t_start_aberr, 
    t_end_aberr,
    index=1
)

In [ ]:
CamHex Timelines

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="CamHex Actual Position", 
    dpi=120,
    figsize=(14, 10),
    tight_layout = True,
    sharex=False,
)

cols = "xyz"
for ax, col in zip("ABC", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s"],
        names=["Actual Position", "Compensated", "Uncompensated"]
    )

for ax, col in zip("DEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos_aberr, cpos_aberr, upos_aberr], 
        column=col, 
        elevation=el_aberr, 
        symbols=["C0o-", "C1x", "C2*"],
        names=["Actual Position with Aberr", "Compensated with Aberr", "Uncompensated with Aberr"]
    )
# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
     
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.7, -0.7), ncol=1)


fig.suptitle("CamHex Timeline - Without and with Added Aberrations")
fig.autofmt_xdate()
fig.tight_layout()

CamHex Positions without and with Aberrations vs MTAOS corrections

In [ ]:
# set width of bar
barWidth = 0.25
plt.figure(figsize=(16,10))
plt.subplots_adjust(hspace=0.3)

for track,aberr in zip(tracks,aberrations):
    ax = plt.subplot(2,2,track+1)
    
    plt.title(f"{test_exec} CamHex Position Difference vs MTAOS corrections")
    
    pos_track = pos[pos.index < move_without[track]].iloc[-10]
    pos_aberr_track = pos_aberr[pos_aberr.index < move_withaberr[track]].iloc[-10]

    cols = ['x', 'y','z','u','v','w']
    pos_difference = pos_aberr_track[cols] - pos_track[cols]
    
    ind = np.arange(6)
    
    plt.bar(ind, pos_difference.values, color ='blue', width = barWidth,
        edgecolor ='grey', label ='Position difference')
    plt.bar([x+barWidth for x in ind], cam_hexapod_correction_applied_xyz.iloc[track*reset], color='orange', 
            width = barWidth, label = "CamHex MTAOS corrections")
    
    ax.set_xticks(ind, pos_difference.index)

    ax.set_title(f'CamHex AOS Corrections vs Position Difference \n with 1 um {aberr}', fontsize=11)
    ax.grid(":", alpha=0.3)

    
ax.legend(ncol=1, bbox_to_anchor=(1.7,2.5), fontsize=12)
plt.show()

---
### M2Hex vs MTAOS corrections

Get data

In [ ]:
pos, cpos, upos, el = await get_hex_positions(t_start, t_end, ind=2)
pos_aberr, cpos_aberr, upos_aberr, el_aberr = await get_hex_positions(t_start_aberr, t_end_aberr, ind=2)

Get the MTAOS corrections 


In [ ]:
m2_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "u", "v" , "w"], 
    t_start_aberr, 
    t_end_aberr,
    index=2
)

M2Hex Timeline

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="M2Hex Actual Position", 
    dpi=120,
    figsize=(14, 10),
    tight_layout = True,
    sharex=False,
)

cols = "xyz"
for ax, col in zip("ABC", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s"],
        names=["Actual Position", "Compensated", "Uncompensated"]
    )

for ax, col in zip("DEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos_aberr, cpos_aberr, upos_aberr], 
        column=col, 
        elevation=el_aberr, 
        symbols=["C0o-", "C1x", "C2*"],
        names=["Actual Position with Aberr", "Compensated with Aberr", "Uncompensated with Aberr"]
    )
# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    

    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.7, -0.7), ncol=1)


fig.suptitle("M2Hex Timeline - Without and with Added Aberrations")
fig.autofmt_xdate()
fig.tight_layout()

M2Hex Positions without and with Aberrations vs MTAOS corrections

In [ ]:
# set width of bar
barWidth = 0.25
plt.figure(figsize=(16,10))
plt.subplots_adjust(hspace=0.3)

for track,aberr in zip(tracks,aberrations):
    ax = plt.subplot(2,2,track+1)
    
    #plt.title(f"{test_exec} M2 Position Difference vs MTAOS corrections")
    
    pos_track = pos[pos.index < move_without[track]].iloc[-3]
    pos_aberr_track = pos_aberr[pos_aberr.index < move_withaberr[track]].iloc[-3]

    cols = ['x', 'y','z','u','v','w']
    pos_difference = pos_aberr_track[cols] - pos_track[cols]
    
    ind = np.arange(6)
    
    plt.bar(ind, pos_difference.values, color ='blue', width = barWidth,
        edgecolor ='grey', label ='Position difference')
    plt.bar([x+barWidth for x in ind], m2_hexapod_correction_applied_xyz.iloc[track*reset], color='orange', 
            width = barWidth, label = "M2 MTAOS corrections")
    
    ax.set_xticks(ind, pos_difference.index)

    ax.set_title(f'M2Hex AOS Corrections vs Position Difference \n with 1 um {aberr}', fontsize=11)
    ax.grid(":", alpha=0.3)

    
ax.legend(ncol=1, bbox_to_anchor=(1.6,2.5), fontsize=12)
plt.show()
        